thử thêm 3 features -> 39 features
3 features về năng lượng.

increase size of matrix.

In [309]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
import noisereduce as nr
import sounddevice as sd
import soundfile as sf
import pickle

In [310]:
# def noise_cancel(filename, classs, dirr):
#     data, fs = librosa.load(os.path.join(dirr,filename))
#     reduced_noise = nr.reduce_noise(audio_clip=data, noise_clip=data)
#     sf.write('./normal_word/' + classs + '/' + filename, data=reduced_noise, samplerate=fs)
# def normal_data():
#     class_names = ["nguoi", "toi", "khong", "mot" , "test_khong" , "test_toi" , "test_mot", "test_nguoi" ]
#     for cname in class_names:
#         files = os.listdir(os.path.join("word", cname))
#         dirfile = os.path.join("word", cname)
#         for f in files:
#             if f.endswith(".wav"):
#                 noise_cancel(f, cname, dirfile)
# normal_data()

In [311]:

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    print('data_dir: ', data_dir)
    print(f'mfcc.shape: {np.array(mfcc).shape}')
    return mfcc

def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans  


In [312]:

class_names = ["nguoi", "toi", "khong", "mot" , "test_khong" , "test_toi" , "test_mot", "test_nguoi" ]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("normal_word", cname))


Load nguoi dataset
data_dir:  normal_word/nguoi
mfcc.shape: (76,)
Load toi dataset
data_dir:  normal_word/toi
mfcc.shape: (76,)
Load khong dataset
data_dir:  normal_word/khong
mfcc.shape: (76,)
Load mot dataset
data_dir:  normal_word/mot
mfcc.shape: (76,)
Load test_khong dataset
data_dir:  normal_word/test_khong
mfcc.shape: (25,)
Load test_toi dataset
data_dir:  normal_word/test_toi
mfcc.shape: (25,)
Load test_mot dataset
data_dir:  normal_word/test_mot
mfcc.shape: (24,)
Load test_nguoi dataset
data_dir:  normal_word/test_nguoi
mfcc.shape: (25,)


In [313]:
# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
print("vectors", all_vectors.shape)

vectors (9303, 36)


In [314]:
for k, v in dataset.items():
    print(v)

[array([[ 103.24,   16.84,   34.16,  -38.18,    6.08,   33.63,   34.98,
           3.34,   28.31,   15.  ,    5.4 ,   -7.52,  -12.4 ,   -1.85,
          -3.69,    8.61,   -0.45,   -8.01,   -4.72,    1.77,   -3.83,
          -1.17,   -0.72,    3.46,   -2.52,    0.86,    4.6 ,    1.61,
           1.96,    1.34,    3.32,    4.1 ,   -0.45,    0.3 ,    1.66,
           2.01],
       [  93.71,    0.95,   -7.76,  -47.95,  -14.3 ,   30.82,   18.41,
         -24.95,   14.8 ,    3.54,    7.76,  -16.97,  -12.4 ,   -1.85,
          -3.69,    8.61,   -0.45,   -8.01,   -4.72,    1.77,   -3.83,
          -1.17,   -0.72,    3.46,   -2.52,    0.86,    4.6 ,    1.61,
           1.96,    1.34,    3.32,    4.1 ,   -0.45,    0.3 ,    1.66,
           2.01],
       [  76.11,  -21.04,   -8.01,  -37.73,  -25.61,    8.84,   14.21,
         -24.16,   17.76,    2.73,  -12.73,  -11.37,  -12.4 ,   -1.85,
          -3.69,    8.61,   -0.45,   -8.01,   -4.72,    1.77,   -3.83,
          -1.17,   -0.72,    3.46,   -2.

[array([[ 52.86, -31.83, -28.54, -12.54, -21.05,  -8.3 ,  -9.53,   7.65,
          1.52, -16.88,   9.51,  -6.01, -10.66,   6.92,   3.28,  -0.79,
          1.32,  -1.25,   1.24,  -2.6 ,  -0.25,   1.98,  -3.52,   0.29,
         -1.45,  -5.53,  -1.66,  -0.35,   1.4 ,   1.25,  -0.29,   1.31,
          1.78,   0.6 ,  -0.09,  -0.64],
       [ 59.54, -19.61, -26.  ,  -3.71, -12.81,  12.99,  -9.77,   6.92,
          5.82,   8.54,  25.93,   0.53, -10.66,   6.92,   3.28,  -0.79,
          1.32,  -1.25,   1.24,  -2.6 ,  -0.25,   1.98,  -3.52,   0.29,
         -1.45,  -5.53,  -1.66,  -0.35,   1.4 ,   1.25,  -0.29,   1.31,
          1.78,   0.6 ,  -0.09,  -0.64],
       [  3.93,  10.23,   4.6 ,  -0.07,   1.  ,   7.75,  -7.77,   6.19,
         -9.41,  -2.13,   5.14, -11.44, -10.66,   6.92,   3.28,  -0.79,
          1.32,  -1.25,   1.24,  -2.6 ,  -0.25,   1.98,  -3.52,   0.29,
         -1.45,  -5.53,  -1.66,  -0.35,   1.4 ,   1.25,  -0.29,   1.31,
          1.78,   0.6 ,  -0.09,  -0.64],
       [ 32.

[array([[ 197.36, -101.67,  -30.11,  -35.25,   19.31,   18.32,  -13.15,
         -14.69,   15.79,   31.65,   10.92,   25.08,  -19.4 ,   21.93,
           2.81,    2.29,   -3.43,   -4.95,   -2.09,   -0.52,   -2.55,
          -5.17,   -4.76,   -6.06,   -0.65,    5.37,   -0.64,    0.25,
           0.83,   -2.72,   -0.94,   -0.68,    0.13,   -0.05,    0.14,
          -1.23],
       [ 193.1 ,  -96.87,  -39.78,  -40.37,   -9.74,   32.63,   -6.72,
          -4.77,    9.96,   31.14,   10.  ,   30.88,  -19.4 ,   21.93,
           2.81,    2.29,   -3.43,   -4.95,   -2.09,   -0.52,   -2.55,
          -5.17,   -4.76,   -6.06,   -0.65,    5.37,   -0.64,    0.25,
           0.83,   -2.72,   -0.94,   -0.68,    0.13,   -0.05,    0.14,
          -1.23],
       [ 181.51,  -98.69,  -30.54,  -34.38,  -17.25,   22.5 ,  -15.27,
          -3.76,    9.95,   23.8 ,    6.59,   32.79,  -19.4 ,   21.93,
           2.81,    2.29,   -3.43,   -4.95,   -2.09,   -0.52,   -2.55,
          -5.17,   -4.76,   -6.06,   -0.

[array([[ 233.62,   53.36,   40.17,   13.44,   32.01,   -0.61,  -11.82,
          15.58,   27.51,   -4.89,   -0.8 ,  -10.6 ,  -16.1 ,   -8.33,
         -10.45,    1.57,   -3.9 ,   -0.71,    2.23,   -3.42,   -3.39,
          -1.25,    1.15,    3.72,    7.54,    0.06,   -0.92,    2.17,
           7.18,    0.68,    0.26,    0.24,    1.12,    0.61,    3.05,
          -0.95],
       [ 131.11,   53.72,   49.05,    7.7 ,  -28.68,  -11.9 ,  -18.97,
          -2.76,   27.07,    1.59,  -15.11,    5.33,  -16.1 ,   -8.33,
         -10.45,    1.57,   -3.9 ,   -0.71,    2.23,   -3.42,   -3.39,
          -1.25,    1.15,    3.72,    7.54,    0.06,   -0.92,    2.17,
           7.18,    0.68,    0.26,    0.24,    1.12,    0.61,    3.05,
          -0.95],
       [ 110.75,   45.84,   33.09,    4.85,  -47.21,  -16.26,  -16.58,
           2.65,   10.35,   -5.78,  -28.55,   14.4 ,  -16.1 ,   -8.33,
         -10.45,    1.57,   -3.9 ,   -0.71,    2.23,   -3.42,   -3.39,
          -1.25,    1.15,    3.72,    7.

[array([[  62.5 ,  -64.7 ,    2.5 ,  -13.5 ,   -3.  ,    9.14,    1.11,
          -1.46,   23.77,   -4.84,    6.51,    7.07,  -12.25,   10.36,
           3.44,    0.67,   -2.26,   -2.6 ,    1.86,    0.46,   -0.69,
           1.41,   -1.51,   -0.06,   -6.87,   -5.57,   -0.39,    0.25,
           1.77,   -0.13,   -0.43,    0.95,    3.17,    1.12,    2.56,
           0.56],
       [ 122.08,  -38.84,  -13.82,    1.48,  -12.18,   40.35,  -12.68,
         -18.86,   11.32,   -0.64,   17.66,    5.39,  -12.25,   10.36,
           3.44,    0.67,   -2.26,   -2.6 ,    1.86,    0.46,   -0.69,
           1.41,   -1.51,   -0.06,   -6.87,   -5.57,   -0.39,    0.25,
           1.77,   -0.13,   -0.43,    0.95,    3.17,    1.12,    2.56,
           0.56],
       [ 106.45,    4.72,   10.77,    6.35,   -7.36,   19.47,   -5.42,
          -6.95,  -14.96,  -26.7 ,   -7.98,   -1.78,  -12.25,   10.36,
           3.44,    0.67,   -2.26,   -2.6 ,    1.86,    0.46,   -0.69,
           1.41,   -1.51,   -0.06,   -6.

[array([[103.19,  31.33,  22.23,  40.25,  23.49,   6.95,   3.21,  -8.64,
        -16.27, -17.39,  -0.28, -14.26, -10.27,   1.  ,   5.  ,  -5.91,
         -6.38,  -7.1 ,  -3.82,  -0.53,   3.93,   3.8 ,  -0.78,   1.08,
          4.62,   1.68,   3.09,  -3.51,  -1.28,  -3.  ,   0.14,   0.02,
          1.08,   0.69,  -0.19,   0.12],
       [ 88.88,  12.13, -11.71,  48.07,  24.67,   9.18,  -4.54,  -1.2 ,
        -18.3 ,  -4.68,  -5.11, -19.37, -10.27,   1.  ,   5.  ,  -5.91,
         -6.38,  -7.1 ,  -3.82,  -0.53,   3.93,   3.8 ,  -0.78,   1.08,
          4.62,   1.68,   3.09,  -3.51,  -1.28,  -3.  ,   0.14,   0.02,
          1.08,   0.69,  -0.19,   0.12],
       [ 70.5 ,  15.06,  -3.2 ,  49.69,  14.76,   6.7 ,  -5.66,   3.62,
        -16.05, -14.41, -11.35, -19.81, -10.27,   1.  ,   5.  ,  -5.91,
         -6.38,  -7.1 ,  -3.82,  -0.53,   3.93,   3.8 ,  -0.78,   1.08,
          4.62,   1.68,   3.09,  -3.51,  -1.28,  -3.  ,   0.14,   0.02,
          1.08,   0.69,  -0.19,   0.12],
       [ 42.

[array([[ 39.22,  64.3 ,  47.17, -17.86,  -2.02,  -0.48,   0.84,   9.51,
         25.1 ,  -7.51,   6.5 , -16.35,  -5.88,  -5.57,  -3.58,  -4.04,
          0.84,   5.19,  -1.05,  -5.38,  -6.04,   1.52,   1.67,   4.54,
          7.12,   4.09,  -0.3 ,  -1.56,  -0.8 ,  -2.57,  -1.33,  -0.27,
          2.64,   2.95,   0.72,  -0.86],
       [-29.27,  29.34,  53.36,  -0.64, -12.58,   7.13,   5.14,  23.24,
          7.14, -10.35,  -0.64, -15.77,  -5.88,  -5.57,  -3.58,  -4.04,
          0.84,   5.19,  -1.05,  -5.38,  -6.04,   1.52,   1.67,   4.54,
          7.12,   4.09,  -0.3 ,  -1.56,  -0.8 ,  -2.57,  -1.33,  -0.27,
          2.64,   2.95,   0.72,  -0.86],
       [-50.22,  16.19,  52.64,   8.62,  -6.24,   8.98,   6.08,  21.86,
          1.4 , -11.5 ,  -9.28, -25.85,  -5.88,  -5.57,  -3.58,  -4.04,
          0.84,   5.19,  -1.05,  -5.38,  -6.04,   1.52,   1.67,   4.54,
          7.12,   4.09,  -0.3 ,  -1.56,  -0.8 ,  -2.57,  -1.33,  -0.27,
          2.64,   2.95,   0.72,  -0.86],
       [-70.

In [315]:
# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors, n_clusters=21)
print("centers", kmeans.cluster_centers_.shape)

centers (21, 36)
centers (21, 36)


In [316]:
kmeans.cluster_centers_

array([[  36.02,   33.7 ,    7.67,  -20.24,  -30.68,   -9.17,    7.19,
          -5.2 ,   -3.35,   -0.76,   -6.12,   -2.05,  -13.02,    1.64,
           0.26,   -0.03,    0.88,   -1.58,    0.76,    0.51,   -0.1 ,
          -1.1 ,   -0.33,   -0.57,   -1.61,   -2.68,   -0.94,    1.31,
           2.47,    1.13,   -0.62,    0.43,    0.33,    0.22,    0.69,
           0.46],
       [  -5.64,   10.66,   -6.03,  -14.16,   -4.55,    1.77,    1.89,
          -9.27,   -2.05,    2.54,    0.22,   -1.06,   -4.16,   -1.94,
          -1.74,    1.58,    1.14,   -2.11,   -0.6 ,    0.64,    0.29,
          -0.47,   -0.17,    0.17,    0.93,   -0.81,    0.38,    1.14,
           0.52,   -0.1 ,   -0.17,    1.  ,    0.25,   -0.24,    0.03,
           0.32],
       [-140.44,  -44.41,    2.64,    7.12,   10.19,   10.44,    4.21,
          14.21,    5.78,    4.94,    7.92,    4.73,    3.26,   -0.29,
          -1.47,   -0.26,   -1.2 ,    1.18,   -0.46,   -0.19,    0.11,
           0.52,    0.76,    0.  ,    2.0

In [317]:
for key, val in dataset.items():
    print(key,'\n', np.array(val).shape)


nguoi 
 (76,)
toi 
 (76,)
khong 
 (76,)
mot 
 (76,)
test_khong 
 (25,)
test_toi 
 (25,)
test_mot 
 (24,)
test_nguoi 
 (25,)


In [318]:
models = {}
original_dataset = {}

In [319]:
original_dataset['nguoi'] = dataset['nguoi'].copy()

In [320]:
cname = 'nguoi'
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1
dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in original_dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=9, random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_=np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
hmm.transmat_=np.array([
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    
])
if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

training class nguoi
(1875, 1) [16, 17, 17, 28, 27, 22, 23, 20, 21, 25, 12, 13, 26, 22, 44, 22, 17, 14, 21, 22, 27, 30, 22, 20, 21, 18, 29, 37, 14, 23, 23, 19, 23, 18, 37, 15, 24, 23, 17, 31, 35, 40, 20, 30, 40, 20, 44, 28, 19, 27, 22, 19, 16, 23, 25, 20, 42, 19, 23, 29, 30, 21, 19, 28, 31, 21, 19, 27, 28, 22, 20, 33, 31, 30, 24, 50] 76


         1       -5732.7613             +nan
         2       -4252.3309       +1480.4304
         3       -3906.7595        +345.5714
         4       -3750.8216        +155.9379
         5       -3681.2010         +69.6206
         6       -3630.8610         +50.3400
         7       -3584.2964         +46.5646
         8       -3532.3252         +51.9713
         9       -3483.0015         +49.3237
        10       -3448.1808         +34.8207
        11       -3421.0925         +27.0883
        12       -3412.5189          +8.5736
        13       -3408.0411          +4.4778
        14       -3404.4874          +3.5537
        15       -3403.0894          +1.3980
        16       -3401.9108          +1.1786
        17       -3400.1420          +1.7688
        18       -3398.4582          +1.6839
        19       -3398.0872          +0.3710
        20       -3397.9909          +0.0962
        21       -3397.9037          +0.0872
        22       -3397.4442          +0.4595
        23

In [321]:
np.set_printoptions(precision=2, suppress=True)
print(models['nguoi'].transmat_)


[[0.89 0.1  0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.81 0.01 0.18 0.   0.   0.   0.   0.  ]
 [0.   0.   0.59 0.   0.41 0.   0.   0.   0.  ]
 [0.   0.   0.   0.96 0.   0.04 0.   0.   0.  ]
 [0.   0.   0.   0.   0.82 0.18 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.78 0.03 0.19 0.  ]
 [0.   0.   0.   0.   0.   0.   0.96 0.04 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.78 0.22]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


In [322]:
original_dataset['toi'] = dataset['toi'].copy()
cname = 'toi'
class_vectors = dataset[cname]
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1

dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in original_dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components= 9, random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
hmm.transmat_ = np.array([ 
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])

if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

training class toi
(1736, 1) [17, 16, 18, 24, 10, 21, 14, 17, 28, 25, 16, 16, 25, 33, 15, 24, 37, 12, 27, 23, 27, 17, 16, 35, 20, 15, 14, 38, 32, 37, 33, 43, 34, 17, 29, 19, 20, 20, 17, 23, 15, 22, 17, 15, 42, 33, 27, 27, 26, 32, 14, 18, 32, 16, 14, 15, 30, 18, 23, 41, 13, 16, 17, 20, 14, 12, 47, 25, 21, 27, 15, 24, 14, 39, 13, 18] 76


         1       -5295.8492             +nan
         2       -4026.3554       +1269.4938
         3       -3702.1865        +324.1689
         4       -3578.6948        +123.4916
         5       -3504.1378         +74.5570
         6       -3426.0846         +78.0532
         7       -3356.2105         +69.8740
         8       -3314.0625         +42.1480
         9       -3291.1242         +22.9383
        10       -3280.2383         +10.8858
        11       -3273.4427          +6.7957
        12       -3267.6929          +5.7497
        13       -3263.3217          +4.3712
        14       -3257.4571          +5.8646
        15       -3252.6738          +4.7833
        16       -3250.9650          +1.7088
        17       -3250.3268          +0.6382
        18       -3249.9799          +0.3469
        19       -3249.7209          +0.2591
        20       -3249.3674          +0.3535
        21       -3248.6560          +0.7114
        22       -3247.7983          +0.8577
        23

In [323]:
np.set_printoptions(precision=2, suppress=True)
print(models['toi'].transmat_)

[[0.46 0.   0.54 0.   0.   0.   0.   0.   0.  ]
 [0.   0.95 0.02 0.03 0.   0.   0.   0.   0.  ]
 [0.   0.   0.48 0.   0.52 0.   0.   0.   0.  ]
 [0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.75 0.24 0.01 0.   0.  ]
 [0.   0.   0.   0.   0.   0.78 0.07 0.15 0.  ]
 [0.   0.   0.   0.   0.   0.   0.95 0.   0.05]
 [0.   0.   0.   0.   0.   0.   0.   0.85 0.15]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


In [324]:
original_dataset['khong'] = dataset['khong'].copy()
cname = 'khong'
class_vectors = dataset[cname]
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1
dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=9, random_state=0, n_iter=1000, verbose=True,
    startprob_prior=np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
    transmat_prior=np.array([ 
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])
)

if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

training class khong
(1898, 1) [27, 17, 27, 27, 33, 21, 20, 23, 29, 45, 25, 26, 27, 33, 35, 20, 30, 20, 28, 16, 22, 21, 15, 28, 24, 17, 22, 24, 18, 26, 22, 23, 25, 33, 25, 14, 33, 21, 20, 27, 23, 19, 20, 22, 25, 19, 30, 31, 18, 20, 21, 19, 20, 28, 25, 17, 26, 20, 38, 40, 25, 27, 25, 34, 34, 20, 24, 23, 23, 28, 27, 25, 26, 36, 32, 19] 76


         1       -5698.4345             +nan
         2       -5199.4249        +499.0096
         3       -5186.3992         +13.0257
         4       -5164.4021         +21.9971
         5       -5117.9487         +46.4534
         6       -5005.9835        +111.9652
         7       -4762.5500        +243.4335
         8       -4370.4984        +392.0516
         9       -3955.9508        +414.5475
        10       -3656.1893        +299.7616
        11       -3470.2186        +185.9707
        12       -3352.2124        +118.0062
        13       -3293.1799         +59.0325
        14       -3258.6914         +34.4885
        15       -3224.0967         +34.5947
        16       -3161.3905         +62.7062
        17       -3116.5388         +44.8517
        18       -3103.8574         +12.6814
        19       -3098.1519          +5.7055
        20       -3094.0589          +4.0930
        21       -3087.8316          +6.2273
        22       -3086.4098          +1.4218
        23

In [325]:
original_dataset['mot'] = dataset['mot'].copy()
cname = 'mot'
# convert all vectors to the cluster index
# dataset['one'] = [O^1, ... O^R] , O^r: the r-th recorded wav file 
# O^r = (c1, c2, ... ct, ... cT) , c_i: the i-th frame in the r-th observation ( or the r-th wav file )
# O^r size T x 1
dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in original_dataset[cname]])


hmm = hmmlearn.hmm.MultinomialHMM(
    n_components=9, random_state=0, n_iter=1000, verbose=True,
    params='te',
    init_params='e'
)
hmm.startprob_=np.array([0.7,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0])
hmm.transmat_=np.array([ 
    [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
    ])
if cname[:4] != 'test':
    X = np.concatenate(dataset[cname])
    lengths = list([len(x) for x in dataset[cname]])
    print("training class", cname)
    print(X.shape, lengths, len(lengths))
    hmm.fit(X, lengths=lengths)
    models[cname] = hmm

         1       -4535.3296             +nan


training class mot
(1472, 1) [22, 23, 18, 11, 24, 20, 18, 18, 18, 26, 24, 16, 36, 16, 25, 19, 21, 17, 20, 20, 16, 20, 27, 19, 23, 23, 20, 12, 14, 18, 10, 18, 22, 21, 36, 16, 23, 23, 16, 18, 22, 26, 18, 12, 16, 19, 15, 15, 12, 18, 11, 20, 24, 14, 18, 18, 20, 14, 23, 22, 15, 20, 23, 17, 17, 15, 16, 21, 23, 20, 21, 21, 24, 16, 23, 16] 76


         2       -3399.8005       +1135.5290
         3       -3081.3549        +318.4457
         4       -2951.8591        +129.4958
         5       -2877.6552         +74.2038
         6       -2842.5186         +35.1366
         7       -2824.3782         +18.1404
         8       -2814.5142          +9.8640
         9       -2807.6727          +6.8416
        10       -2800.4429          +7.2297
        11       -2789.2260         +11.2169
        12       -2774.9942         +14.2319
        13       -2762.6133         +12.3809
        14       -2752.1943         +10.4190
        15       -2742.4770          +9.7173
        16       -2736.0518          +6.4252
        17       -2731.0507          +5.0011
        18       -2728.2171          +2.8336
        19       -2727.0395          +1.1776
        20       -2726.4821          +0.5574
        21       -2726.0356          +0.4465
        22       -2725.5360          +0.4996
        23       -2724.9712          +0.5648
        24

In [326]:
pickle.dump(models, open('./model/models.pk','wb'))

In [327]:
dataset["test_nguoi"] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_nguoi']])
dataset['test_toi'] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_toi']])
dataset['test_khong'] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_khong']])
dataset['test_mot'] = list([kmeans.predict(v).reshape(-1,1) for v in dataset['test_mot']])

In [328]:
print("Testing")
mapping = ["nguoi", "toi", "khong", "mot"]
class_names = ["test_nguoi", "test_toi", "test_khong", "test_mot"]
count = 0
correct = 0
for true_cname in class_names:
    score = []
    for i in dataset[true_cname]:
        score = [model.score(i, [len(i)]) for cname, model in models.items() if cname[:4] != 'test']
        res = mapping[score.index(max(score))] 
        if res == true_cname[5:]:
            correct += 1
        count += 1 
print(100*correct/count)

Testing
76.76767676767676
